In [ ]:
%run style_and_packages.ipynb

# Beispiel: Antwortspektrenverfahren an einem Rahmen


## Aufgabenstellung

Dies ist eine Weiterführung des bereits bekannten Rahmentragwerks aus @sec-mms_rahmen.

![Statisches System des Rahmentragwerks](bilder/mms5.jpg){#fig-system width=70%}



Gesucht:

-  Eigenkreisfrequenz $\omega$ 
-  Eigenformen - Normierung auf 
$$\phi_1^T = 
\begin{bmatrix}
 &  1\\
\end{bmatrix} $$
$$\phi_2^T =
\begin{bmatrix}
 &  1\\
\end{bmatrix}$$

- Skizze der Eigenformen 
- Statische Ersatzkräfte mit elastischem Antwortspektrum aus [@SIA261_2020] Abs. 16.2.3 auf Stockwerksebene. Überlagerung mit der SRSS-Methode.

Gegeben:

- Elastizitätsmodul der Stützen $E = 30000 \text{N}/{mm}^2$
- Flächenträgheitsmoment der Stützen $I = 2 \cdot 10^9 \text{mm}^4$
- Dehnsteifigkeit aller Stäbe $E\cdot A = \infty$
- Einzelmassen $m = 20000 \text{kg}$
- Baugrundklasse B
- Erdbebenzone Z2

{{< pagebreak >}}

## Musterlösung 

[Bei dieser Musterlösung bin ich nicht ganz sattelfest, müssen wir zusammen besprechen Stephan]{.underline}

In [51]:
#| label: tbl-parameter_mms5
#| tbl-cap: Verwendete Parameter

params = {'E': 30*10**3 *unit.N/unit.mm**2,
          'I':2*10**9*unit.mm**4,
          'm_1':2*20000*unit.N*unit.second**2/unit.m,
          'm_2':20000*unit.N*unit.second**2/unit.m,
          'H':3.2*unit.m,
          
          }
render.dict_to_table(params)

| Parameter |  |
|---|---|
| $E = \frac{30000 \text{N}}{\text{mm}^{2}}$ | $H = 3.2 \text{m}$ |
| $I = 2000000000 \text{mm}^{4}$ | $m_{1} = \frac{40000 \text{N} \text{s}^{2}}{\text{m}}$ |
| $m_{2} = \frac{20000 \text{N} \text{s}^{2}}{\text{m}}$ | |


In [52]:
omega, t,  l, m_1, m_2, E,I, H = sp.symbols('omega, t,  l, m_1, m_2, E, I, H')

omega_n = sp.symbols('omega_n')

### Steifigkeitsmatrix $\mathbf{K}$

Zur Bestimmung der Steifigkeitsmatrix ist das System an jedem Freiheitsgrad auszulenken, wie in @fig-steifigkeit dargestellt ist.

![Auslenkung der Freiheitsgrade zur Bestimmung der Steifigkeit](bilder/steifigkeit_mms2.jpg){#fig-steifigkeit}

Wichtig dabei sind die Richtungen der Kräfte. Als Denkstütze gilt folgendes:
- Der Auslenkung um $u$ wirkt die Federkraft entgegen, welche $k u$ entspricht.
- Zusätzlich wirkt die Trägheitskraft der Auslenkung entgegen, welche $m u''$ entspricht.
- Nach der Betrachtung des ausgelenkten Punkts, kann mittels *Actio-Reactio*-Prinzip das "*Stockwerk*" ins Gleichgewicht gebracht werden.
- Vorzeichen sind gegen der Bewegungsrichtig positiv.

#### Horizontale Steifigkeit

Die horizontale Steifigkeit des Systems lässt sich mittels der Formel für Rahmen bestimmen.

$$k_{Stütze} = \frac{12EI_{Stütze}}{H^3}$$

Diese gilt für eine einzelne Stütze.

In [53]:
k_1, k_2 = sp.symbols('k_1, k_2')

params['k_1'] = 2*(12 * 2*E*I / H**3).subs(params).simplify().evalf(6)
params['k_2'] = 2*(12 * E*I / H**3).subs(params).simplify().evalf(6)
K = sp.Matrix([[k_1 + k_2, -k_2],[-k_2, k_2]])


render.eq_display(sp.MatrixSymbol('K', 2,2), K,
                  sp.MatrixSymbol('K', 2,2), K.subs(params),
                  )

    ⎡k₁ + k₂  -k₂⎤
K = ⎢            ⎥
    ⎣  -k₂    k₂ ⎦

    ⎡ 1.31836e+8⋅newton   -4.39453e+7⋅newton ⎤
    ⎢ ─────────────────   ───────────────────⎥
    ⎢       meter                meter       ⎥
K = ⎢                                        ⎥
    ⎢-4.39453e+7⋅newton    4.39453e+7⋅newton ⎥
    ⎢───────────────────   ───────────────── ⎥
    ⎣       meter                meter       ⎦

### Eigenvektoren
#### Massenmatrix $\mathbf{M}$ 

Die Massenmatrix folgt dem gleichen Aufbau wie die Steifigkeitsmatrix. Es gelten die gleichen Vorzeichenregelungen.

In [54]:
m_1, m_2 = sp.symbols('m_1, m_2')
M = sp.Matrix([[m_1, 0],[0, m_2]])

render.eq_display(sp.MatrixSymbol('M', 2,2), M,
                  sp.MatrixSymbol('M', 2,2), M.subs(params))

    ⎡m₁  0 ⎤
M = ⎢      ⎥
    ⎣0   m₂⎦

    ⎡                   2                      ⎤
    ⎢40000⋅newton⋅second                       ⎥
    ⎢────────────────────           0          ⎥
    ⎢       meter                              ⎥
M = ⎢                                          ⎥
    ⎢                                         2⎥
    ⎢                      20000⋅newton⋅second ⎥
    ⎢         0            ────────────────────⎥
    ⎣                             meter        ⎦

#### Eigenkreisfrequenzen 
Bei einem Mehrmassenschwinger gibt es entsprechend den Freiheitsgraden Eigenkreisfrequenzen $\omega_n$. Diese lassen sich anhand folgender Gleichung bestimmen:

$$\det{[\mathbf{K}-\omega_n^2 \mathbf{M}]=0}$$

In [55]:
eq_omega = sp.det(K-omega_n**2*M)

omega_n_solve = sp.solve(eq_omega, omega_n)
omega_1 = omega_n_solve[1]
omega_2 = omega_n_solve[3]

render.eq_display('omega_1', omega_1.subs(params).simplify().evalf(3),
                  'omega_2', omega_2.subs(params).simplify().evalf(3))

      33.1 
ω₁ = ──────
     second

      66.3 
ω₂ = ──────
     second

#### Eigenvektoren $\phi$

In [56]:
phi_11, phi_21, phi_12, phi_22 = sp.symbols('phi_11, phi_21, phi_12, phi_22')
params['phi_21'] = 1

phi_1 = sp.Matrix([[phi_11], [phi_21]])

phi_11 = list(sp.solve((K-omega_1**2 *M)*phi_1, phi_11).values())[0]

params['phi_11'] = phi_11.subs(params).simplify() 




render.eq_display(sp.simplify((K-omega_1**2 *M)*phi_1),sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_1', 2,1),sp.simplify(phi_1.subs(params)).evalf(3))

⎡                 ⎛                           ________________________________
⎢                 ⎜                          ╱   2   2                        
⎢             φ₁₁⋅⎝-k₂⋅m₁ + m₂⋅(k₁ + k₂) + ╲╱  k₁ ⋅m₂  - 2⋅k₁⋅k₂⋅m₁⋅m₂ + 2⋅k₁⋅
⎢-k₂⋅m₂⋅φ₂₁ + ────────────────────────────────────────────────────────────────
⎢                                                                 2           
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                           m₂                
⎢                                                                             
⎢                 ⎛                          _________________________________
⎢                 ⎜                         ╱   2   2                         
⎢             φ₂₁⋅⎝k₂⋅m₁ - m₂⋅(k₁ + k₂) + ╲╱  k₁ ⋅m₂  - 2⋅k₁⋅k₂⋅m₁⋅m₂ + 2⋅k₁⋅k
⎢-k₂⋅m₁⋅φ₁₁ + ────────────────────────────────────────────────────────────────
⎢                                                   

     ⎡0.5⎤
φ₁ = ⎢   ⎥
     ⎣1.0⎦

In [57]:
params['phi_22'] = 1

phi_2 = sp.Matrix([[phi_12], [phi_22]])

phi_12 = list(sp.solve((K-omega_2**2 *M)*phi_2, phi_12).values())[0]

params['phi_12'] = phi_12.subs(params).simplify() 




render.eq_display(sp.simplify((K-omega_2**2 *M)*phi_2),sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_2', 2,1),sp.simplify(phi_2.subs(params)).evalf(3))

⎡                 ⎛                           ________________________________
⎢                 ⎜                          ╱   2   2                        
⎢             φ₁₂⋅⎝-k₂⋅m₁ + m₂⋅(k₁ + k₂) - ╲╱  k₁ ⋅m₂  - 2⋅k₁⋅k₂⋅m₁⋅m₂ + 2⋅k₁⋅
⎢-k₂⋅m₂⋅φ₂₂ + ────────────────────────────────────────────────────────────────
⎢                                                                 2           
⎢─────────────────────────────────────────────────────────────────────────────
⎢                                                           m₂                
⎢                                                                             
⎢                 ⎛                          _________________________________
⎢                 ⎜                         ╱   2   2                         
⎢             φ₂₂⋅⎝k₂⋅m₁ - m₂⋅(k₁ + k₂) - ╲╱  k₁ ⋅m₂  - 2⋅k₁⋅k₂⋅m₁⋅m₂ + 2⋅k₁⋅k
⎢-k₂⋅m₁⋅φ₁₂ + ────────────────────────────────────────────────────────────────
⎢                                                   

     ⎡-1.0⎤
φ₂ = ⎢    ⎥
     ⎣1.0 ⎦

#### Orthogonalitätsbedingung

Zur effektiven Entkoppelung der Gleichungen muss die Orthogonalitätsbedingung eingehalten sein. Dies gilt es für die Massenmatrix zu kontrollieren:

Dies gilt es für die Massenmatrix zu kontrollieren:

$$\phi_1^T \mathbf{ M } \phi_1 \neq 0$$

$$\phi_2^T \mathbf{ M } \phi_2 \neq 0$$

$$\phi_2^T \mathbf{ M } \phi_1 = 0$$

Sowohl auch für die Steifigkeitsmatrix:

$$\phi_1^T \mathbf{ K } \phi_1 \neq 0$$

$$\phi_2^T \mathbf{ K } \phi_2 \neq 0$$

$$\phi_2^T \mathbf{ K } \phi_1 = 0$$


Angewendet auf die Aufgabe:


In [58]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),(phi_1.T*M*phi_1).subs(params).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_2.T*M*phi_2).subs(params).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),(phi_2.T*M*phi_1).subs(params).evalf(3),

                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_1.T*M*phi_2).subs(params).evalf(3))

           ⎡                    2⎤
  T        ⎢3.0e+4⋅newton⋅second ⎥
φ₁ ⋅M⋅φ₁ = ⎢─────────────────────⎥
           ⎣        meter        ⎦

           ⎡                    2⎤
  T        ⎢6.0e+4⋅newton⋅second ⎥
φ₂ ⋅M⋅φ₂ = ⎢─────────────────────⎥
           ⎣        meter        ⎦

  T           
φ₂ ⋅M⋅φ₁ = [0]

  T           
φ₁ ⋅M⋅φ₂ = [0]

Für die Steifigkeitsmatrix:

In [59]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_1.T*K*phi_1).subs(params)).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_2.T*K*phi_2).subs(params)).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_2.T*K*phi_1).subs(params)).evalf(4),
                  
                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_1.T*K*phi_2).subs(params)).evalf(4))

  T        ⎡3.3e+7⋅newton⎤
φ₁ ⋅K⋅φ₁ = ⎢─────────────⎥
           ⎣    meter    ⎦

  T        ⎡2.64e+8⋅newton⎤
φ₂ ⋅K⋅φ₂ = ⎢──────────────⎥
           ⎣    meter     ⎦

  T           
φ₂ ⋅K⋅φ₁ = [0]

  T           
φ₁ ⋅K⋅φ₂ = [0]

#### Eigenformen

![Die beiden Eigenformen skizziert](bilder/eigenformen_mms2.jpg){#fig-eigenformen}

### Modale Analyse

Die Bewegungsgleichung für einen ungedämpften, frei schwingenden Mehrmassenschwinger lässt sich folgend beschreiben:

$$\mathbf{M u''(t) + K u = 0}$$

Die Matrix-Gleichung beschreibt ein System aus Differentialgleichungen. Die Modale Analyse zielt darauf ab, diese zu entkoppeln. Bezogen auf den Mehrmassenschwinger heisst eine Entkoppelung, dass diese in Einmassenschwinger aufgeteilt werden. Dies wird nun schrittweise durchgeführt.

#### Modal- und Spektralmatrix

Mittels der Modal- und Spektralmatrix können die generalisierten Grössen ermittelt werden. Diese sind die eigenschaften der einzelnen Einmassenschwinger. Die generalisierten Werte besitzen keine physikalischen Werte, sie sind abhängig von der Wahl der Eigenvektoren, welche bekanntlich von der Normierung abhängen.


Aufgrund der Bewegungsgleichung können die generalisierten Grössen bestimmt werden, es gilt:

$$\mathbf{\Phi^T M \Phi u''(t) + \Phi^T K \Phi u(t) = 0}$$

$$\mathbf{M^*u''(t) + K^* u(t) = 0}$$

Alle $N$-Eigenwerte und alle $N$-Eigenvektoren können kompakt
mit Matrizen ausgedrückt werden:


In [60]:
Phi = sp.Matrix([[phi_1, phi_2]])
Omega = sp.Matrix([[omega_1, 0],[0, omega_2]])

render.eq_display('Modalmatrix', 'Phi',
                  sp.MatrixSymbol('Phi', 2, 2), Phi.subs(params).evalf(4),
                  'Spektralmatrix', 'Omega^2',
                  sp.MatrixSymbol('Omega^2', 2, 2), (Omega**2).subs(params).evalf(4))

Modalmatrix = Φ

    ⎡0.5  -1.0⎤
Φ = ⎢         ⎥
    ⎣1.0  1.0 ⎦

                  2
Spektralmatrix = Ω 

     ⎡ 1099.0         ⎤
     ⎢───────     0   ⎥
     ⎢      2         ⎥
     ⎢second          ⎥
Ω² = ⎢                ⎥
     ⎢          4395.0⎥
     ⎢   0     ───────⎥
     ⎢               2⎥
     ⎣         second ⎦

#### Generalisierte Grössen


In [61]:
M_star = Phi.T * M * Phi
K_star = Phi.T * K * Phi


render.eq_display(sp.MatrixSymbol('M^\star', 2,2),sp.simplify(M_star.subs(params)).evalf(5),
                  sp.MatrixSymbol('K^\star', 2,2),sp.simplify(K_star.subs(params)).evalf(5))
                 

           ⎡                     2                        ⎤
           ⎢30000.0⋅newton⋅second                         ⎥
           ⎢──────────────────────            0           ⎥
           ⎢        meter                                 ⎥
M__\star = ⎢                                              ⎥
           ⎢                                             2⎥
           ⎢                        60000.0⋅newton⋅second ⎥
           ⎢          0             ──────────────────────⎥
           ⎣                                meter         ⎦

           ⎡3.2959e+7⋅newton                  ⎤
           ⎢────────────────         0        ⎥
           ⎢     meter                        ⎥
K__\star = ⎢                                  ⎥
           ⎢                  2.6367e+8⋅newton⎥
           ⎢       0          ────────────────⎥
           ⎣                       meter      ⎦

#### Kontrolle der modalen Transformation

In [62]:
omega_1_kontrolle = sp.sqrt(K_star[0] / M_star[0])
omega_2_kontrolle = sp.sqrt(K_star[3] / M_star[3])

render.eq_display('omega_1', omega_1.subs(params).simplify().evalf(3),
                  'omega_1_modal', omega_1_kontrolle.subs(params).simplify().evalf(3),
                  'omega_2', omega_2.subs(params).simplify().evalf(4),
                  'omega_2_modal', omega_2_kontrolle.subs(params).simplify().evalf(4))




      33.1 
ω₁ = ──────
     second

             33.1 
ω_1_modal = ──────
            second

     66.29 
ω₂ = ──────
     second

            66.29 
ω_2_modal = ──────
            second

#### Partizipationsfaktor $\Gamma$

Die Verteilung des Partizipationsfaktor gibt einen direkten Hinweis, welcher Eigenmode an der Gesamtanwort den grössten Einfluss (beteiligt bzw. partizipiert) hat.

$$\Gamma_n = \frac{\Phi_n^T \mathbf{M 1}}{\Phi_n^T \mathbf{M}\Phi_n}$$

In allgemeiner Form lautet der Partizipationsfaktor:

$$\Gamma_n = \frac{\Phi_n^T \mathbf{M r^\star}}{\Phi_n^T \mathbf{M}\Phi_n}$$

$\mathbf{r^\star}$ beschreibt die Starrkörperverschiebung infolge der Erdbebenanregung $u_g$ am Fusspunktb des Gesamtsystems.



$$\mathbf{r^\star} = \begin{bmatrix}
FHG_1 \\
FHG_2 
\end{bmatrix}=
\begin{bmatrix}
\cos(0) \\
\cos(0) 
\end{bmatrix}=
\begin{bmatrix}
1 \\
1 
\end{bmatrix}= \mathbf{1}$$


In [63]:
one_vec = sp.ones(2,1)

gamma_1 = (phi_1.T*M*one_vec)[0]/(phi_1.T*M*phi_1)[0]
gamma_2 = (phi_2.T*M*one_vec)[0]/(phi_2.T*M*phi_2)[0]
gamma_sqr = sp.Matrix([[gamma_1**2],[gamma_2**2]])
render.eq_display('Gamma_1', gamma_1,
                  'Gamma_1', gamma_1.subs(params).evalf(3),
                  'Gamma_2', gamma_2,
                  'Gamma_2', gamma_2.subs(params).evalf(3),
                  sp.MatrixSymbol('Gamma^2_n',2,1),gamma_sqr.subs(params).evalf(3))

      m₁⋅φ₁₁ + m₂⋅φ₂₁ 
Γ₁ = ─────────────────
           2         2
     m₁⋅φ₁₁  + m₂⋅φ₂₁ 

Γ₁ = 1.33

      m₁⋅φ₁₂ + m₂⋅φ₂₂ 
Γ₂ = ─────────────────
           2         2
     m₁⋅φ₁₂  + m₂⋅φ₂₂ 

Γ₂ = -0.333

      ⎡1.78 ⎤
Γ²ₙ = ⎢     ⎥
      ⎣0.111⎦

#### Effektive Modale Massen

Durch Multiplikation der Modalen Massen mit dem Partizipationsfaktor resultieren die effektiven Modalen Massen.

In [64]:
m_1_eff = gamma_1**2 * M_star[0]
m_2_eff = gamma_2**2 * M_star[3]

render.eq_display('m_1eff', m_1_eff.subs(params).evalf(3),
                  'm_2eff', m_2_eff.subs(params).evalf(3),
                  'm_tot', (m_1_eff+m_2_eff).subs(params).evalf(3))


                              2
         5.33e+4⋅newton⋅second 
m_1eff = ──────────────────────
                 meter         

                              2
         6.67e+3⋅newton⋅second 
m_2eff = ──────────────────────
                 meter         

                           2
       6.0e+4⋅newton⋅second 
mₜₒₜ = ─────────────────────
               meter        

#### Modale Höhen

In [65]:

h_1_star = (H*m_1*phi_11 + H*m_2*phi_12)  / (m_1 * phi_11 + m_2 * phi_12)
h_2_star = (H*m_1*phi_21 + H*m_2*phi_22)  / (m_1 * phi_21 + m_2 * phi_22)

render.eq_display("h_1" , "(H*m_1*phi_11 + H*m_2*phi_12)  / (m_1 * phi_11 + m_2 * phi_12)",
'h_1', h_1_star.subs(params),
"h_2" , "(H*m_1*phi_21 + H*m_2*phi_22)  / (m_1 * phi_21 + m_2 * phi_22)",
'h_2', h_2_star.subs(params))

     H⋅m₁⋅φ₁₁ + H⋅m₂⋅φ₁₂
h₁ = ───────────────────
       m₁⋅φ₁₁ + m₂⋅φ₁₂  

False

     H⋅m₁⋅φ₂₁ + H⋅m₂⋅φ₂₂
h₂ = ───────────────────
       m₁⋅φ₂₁ + m₂⋅φ₂₂  

h₂ = 3.2⋅meter

### Elastisches Antwortspektrum

Dem Vorgehen nach [@SIA261_2020] Abs. 16.2.3.1 entsprechend, werden folgende Parameter definiert:

- Baugrundklasse B
- Erdbebenzone Z2
- $a_{gd} = 1.0 \text{ m}/\text{s}^2$

In [66]:
T_B = 0.08*unit.second
T_C = 0.35*unit.second
T_D = 2*unit.second
S=1.2
eta = 1.0
a_gd = 1*unit.m/unit.second**2

#### Grundschwingzeit

Die Grundschwingzeit kann anhand der bereits ermittelten Eigenkreisfrequenzen ermittelt werden.

In [67]:
T_1 = (omega_1 /(2*sp.pi))**-1
T_2 = (omega_2 /(2*sp.pi))**-1

render.eq_display('T_1', T_1.subs(params).evalf(3),
                  'T_2', T_2.subs(params).evalf(3),
                  )

T₁ = 0.19⋅second

T₂ = 0.0948⋅second

#### Elastisches Antwortspektrum

In [68]:
def antwortspektrum(a_gd,T, Baugrundklasse='A', eta=1):
    from sympy.physics.units import second

    # Baugrundklassen
    Baugrundklassen = {
        'A': {
            'T_B':0.07*second,
            'T_C':0.25*second,
            'T_D':2.0*second,
            'S':1.0,
        },
        'B': {
            'T_B':0.08*second,
            'T_C':0.35*second,
            'T_D':2.0*second,
            'S':1.2,
        },
        'C': {
            'T_B':0.10*second,
            'T_C':0.4*second,
            'T_D':2.0*second,
            'S':1.45,
        },
        'D': {
            'T_B':0.10*second,
            'T_C':0.5*second,
            'T_D':2.0*second,
            'S':1.7,
        },
        'E': {
            'T_B':0.09*second,
            'T_C':0.25*second,
            'T_D':2.0*second,
            'S':1.7,
        }
    }

    # Gewünschte Baugrundklasse auswählen
    params = Baugrundklassen[Baugrundklasse]
    T_B = params['T_B']
    T_C = params['T_C']
    T_D = params['T_D']
    S = params['S']

    if T<=T_B:
        render.eq_display('S_e','a_gd*S_*(1+(2.5*eta_-1)*T/T_B)')
        return a_gd * S *(1+(2.5*eta-1)*T/T_B)
    
    if T_B<T<=T_C:
        render.eq_display('S_e','2.5*a_gd*S_*eta')
        return 2.5*a_gd*S*eta
    
    if T_C<T<=T_D:
        render.eq_display('S_e', '2.5*a_gd*S_*eta*T_C/T')
        return 2.5*a_gd*S*eta*T_C/T
    
    if T_D<=T:
        render.eq_display('S_e', '2.5*a_gd*S_*eta*T_C*T_D/T**2')
        return 2.5*a_gd*S*eta*T_C*T_D/T**2
    
    else:
        print('Falsche Eingabe')


In [69]:

S_e_1 = antwortspektrum(a_gd, T_1.subs(params).evalf(3), Baugrundklasse='B')
render.eq_display('S_e_1', S_e_1.evalf(3))

Sₑ = 2.5⋅S⋅a_gd⋅η

       3.0⋅meter
Sₑ ₁ = ─────────
              2 
        second  

In [70]:
S_e_2 = antwortspektrum(a_gd, T_2.subs(params).evalf(3), Baugrundklasse='B')

render.eq_display('S_e_2', S_e_2.evalf(3))

Sₑ = 2.5⋅S⋅a_gd⋅η

       3.0⋅meter
Sₑ ₂ = ─────────
              2 
        second  

#### Bemessungsspektrum

Es wird lediglich auf charakteristischem Niveau die Kräfte bestimmt

In [71]:
q = 1.5
yf = 1.5

In [72]:
def bemessungsspektrum(T_B, T_C, T_D, q, yf, S, a_gd,T):
    g = 9.81*unit.m/unit.second**2
    if T<=T_B:
        render.eq_display('S_d','yf*a_gd/g*S_*(0.67+(2.5/q-0.67)*T/T_B)')
        return yf*a_gd/g*S*(0.67+(2.5/q-0.67)*T/T_B)
    
    if T_B<T<=T_C:
        render.eq_display('S_d','2.5*yf*a_gd/g*S_/q')
        return 2.5*yf*a_gd/g*S/q
    
    if T_C<T<=T_D:
        render.eq_display('S_e', '2.5*yf*a_gd/g*S_*T_C/(T*q)')
        return 2.5*yf*a_gd/g*S*T_C/(T*q)
    
    if T_D<=T:
        return max([2.5*yf*a_gd/g*S*T_C*T_D/(T**2*q),0.1*yf*a_gd/g])
    
    else:
        print('Falsche Eingabe')

#### Einwirkungen

In [73]:
f_11 = gamma_1*m_1_eff*phi_11*S_e_1
f_21 = gamma_1*m_2_eff*phi_21*S_e_1

f_12 = gamma_2*m_1_eff*phi_12*S_e_2
f_22 = gamma_2*m_2_eff*phi_22*S_e_2


f_jn = sp.Matrix([[f_11, f_12],[f_12,f_22]])

render.eq_display("f_11 ", "gamma_1*m_1_eff*phi_11*S_e_1",
"f_21" , "gamma_1*m_2_eff*phi_21*S_e_1",
"f_12" , "gamma_2*m_1_eff*phi_12*S_e_2",
"f_22" , "gamma_2*m_2_eff*phi_22*S_e_2",


sp.MatrixSymbol('f_jn', 2,2),f_jn.subs(params).evalf(3))

f₁₁ = Sₑ ₁⋅γ₁⋅m_1_eff⋅φ₁₁

f₂₁ = Sₑ ₁⋅γ₁⋅m_2_eff⋅φ₂₁

f₁₂ = Sₑ ₂⋅γ₂⋅m_1_eff⋅φ₁₂

f₂₂ = Sₑ ₂⋅γ₂⋅m_2_eff⋅φ₂₂

       ⎡1.07e+5⋅newton  5.33e+4⋅newton ⎤
f_jn = ⎢                               ⎥
       ⎣5.33e+4⋅newton  -6.67e+3⋅newton⎦

#### Deformation

In [74]:
u_11 = gamma_1*phi_11*S_e_1/omega_1**2
u_21 = gamma_1*phi_21*S_e_1/omega_1**2
u_12 = gamma_2*phi_12*S_e_2/omega_2**2
u_22 = gamma_2*phi_22*S_e_2/omega_2**2


u_jn = sp.Matrix([[u_11, u_12],[u_12,u_22]])

render.eq_display("u_11" ," gamma_1*phi_11*S_e_1/omega_1**2",
"u_21" , "gamma_1*phi_21*S_e_1/omega_1**2",
"u_12" , "gamma_2*phi_12*S_e_2/omega_2**2",
"u_22" ,"gamma_2*phi_22*S_e_2/omega_2**2",
    
    sp.MatrixSymbol('u_jn', 2,2),unit.convert_to(u_jn.subs(params).evalf(3),unit.mm))

      Sₑ ₁⋅γ₁⋅φ₁₁
u₁₁ = ───────────
            2    
          ω₁     

      Sₑ ₁⋅γ₁⋅φ₂₁
u₂₁ = ───────────
            2    
          ω₁     

      Sₑ ₂⋅γ₂⋅φ₁₂
u₁₂ = ───────────
            2    
          ω₂     

      Sₑ ₂⋅γ₂⋅φ₂₂
u₂₂ = ───────────
            2    
          ω₂     

       ⎡1.82⋅millimeter   0.228⋅millimeter ⎤
u_jn = ⎢                                   ⎥
       ⎣0.228⋅millimeter  -0.228⋅millimeter⎦

### Überlagerung der EMS

Um die Entkoppelung rückzuführen gilt es die erhaltenen Resultate zu überlagern. Dabei gibt es unterschiedliche Ansätze. Bei weit auseinander liegenden Eigenfrequenzen kann die SRSS-Überlagerung verwendet werden.

$$V_{b2max} = \sqrt{\sum_{n=1}^2 f_{j,n}^2}$$

In [75]:
V_b1_max = sp.sqrt(f_11**2 + f_12**2)
V_b2_max = sp.sqrt(f_21**2 + f_22**2)

render.eq_display("V_b1_max" ,"sqrt(f_11**2 + f_12**2)",
'V_b1max',V_b1_max.subs(params).evalf(3),
"V_b2_max" , "sqrt(f_21**2 + f_22**2)",
'V_b2max', V_b2_max.subs(params).evalf(3))

              _____________
             ╱    2      2 
V_b1_max = ╲╱  f₁₁  + f₁₂  

V_b1max = 1.19e+5⋅newton

              _____________
             ╱    2      2 
V_b2_max = ╲╱  f₂₁  + f₂₂  

V_b2max = 2.75e+4⋅newton

Deformationen:

$$u_{bn} = \sqrt{\sum_{n=1}^2 u_{j,n}^2}$$

In [76]:
u_b1 = sp.sqrt(u_11**2 + u_12**2)
u_b2 = sp.sqrt(u_21**2 + u_22**2)

render.eq_display(
"u_b1" , "sqrt(u_11**2 + u_12**2)",
'u_b1',unit.convert_to(u_b1.subs(params).evalf(3), unit.mm),
"u_b2", "sqrt(u_21**2 + u_22**2)",
'u_b2', unit.convert_to(u_b2.subs(params).evalf(3),unit.mm))

          _____________
         ╱    2      2 
u_b1 = ╲╱  u₁₁  + u₁₂  

u_b1 = 1.83⋅millimeter

          _____________
         ╱    2      2 
u_b2 = ╲╱  u₂₁  + u₂₂  

u_b2 = 3.65⋅millimeter

[Skizze mit Schnittgrössen ergänzen]{.underline}